# Install correct packages + Define Wigner Function (run all 5 cells)

In [ ]:
!pip install scipy==1.2.1
!pip install qutip

In [ ]:
#inporting qutip, numpy and plotting
from qutip import *
from numpy import *
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.axes3d import Axes3D
from matplotlib import cm, colors

In [ ]:
# redefining the Wigner function because qutip had a bug in the source code
from scipy.special import *

def _rho_kq(rho, j, k, q):
    v = 0j

    for m1 in arange(-j, j+1):
        for m2 in arange(-j, j+1):
            v += (-1)**(j - m1 - q) * clebsch(j, j, k, m1, -m2,
                                              q) * rho.data[int(m1 + j), int(m2 + j)] #added int

    return v


def spin_wigner(rho, theta, phi):
    """Wigner function for a spin-j system on the 2-sphere of radius j
       (for j = 1/2 this is the Bloch sphere).

    Parameters
    ----------
    state : qobj
        A state vector or density matrix for a spin-j quantum system.
    theta : array_like
        Polar angle at which to calculate the W function.
    phi : array_like
        Azimuthal angle at which to calculate the W function.

    Returns
    -------
    W, THETA, PHI : 2d-array
        Values representing the spin Wigner function at the values specified
        by THETA and PHI.

    Notes
    -----
    Experimental.

    """

    if rho.type == 'bra':
        rho = rho.dag()

    if rho.type == 'ket':
        rho = ket2dm(rho)

    J = rho.shape[0]
    j = (J - 1) / 2

    THETA, PHI = meshgrid(theta, phi)

    W = np.zeros_like(THETA, dtype=complex)

    for k in range(int(2 * j)+1):
        for q in arange(-k, k+1):
            # sph_harm takes azimuthal angle then polar angle as arguments
            W += _rho_kq(rho, j, k, q) * sph_harm(q, k, PHI, THETA)

    return W, THETA, PHI

In [ ]:
# Project from full 4 dimensional space into the J=1 subspace
def symmetric_projection(rho):
  J = 1
  data = [np.sum([[clebsch(1/2, 1/2, J, m1-1/2, m2-1/2, m)*rho.data[int(m1*2+m2)] for m1 in [0,1]] for m2 in [0,1] ]) for m in arange(-J,J+1)]
  rho_spin_1 = basis(2*J+1,0) * data[0]
  for m in range(1,2*J+1):
    rho_spin_1 += basis(2*J+1,m) * data[m]
  return Qobj(rho_spin_1)

In [ ]:
#test projection
symmetric_projection(Qobj(np.array([0,1/np.sqrt(2),1/np.sqrt(2),0]).T)).full

# Single qubit circuit

## Get quantum circuit output
Multiply matricies for single qubit gates as in Section III of the main text

In [ ]:
# intial state
psi_initial = ket("0")
print("Inital State:")
print(psi_initial)

In [ ]:
# Rotate into x direction with Hadamard transformation
H1 = qutip.qip.operations.gates.hadamard_transform(1) #one qubit Hadamard
print("One qubit Hadamard:")
print(H1)

psi_x = H1 * psi_initial #Apply Hadamard by multiplication
print("\nResulting state:")
print(psi_x)

In [ ]:
# Define Rz gate for theta = pi/3
phi = np.pi/3
Rz_phi = qutip.qip.operations.gates.rz(phi)
print("Rz operator:")
print(Rz_phi)
# Apply Rz gate to inital state
psi_theta = Rz_phi * psi_x
print("\nRotated State:")
print(psi_theta)

In [ ]:
# Action of final beamsplitter 
psi_final = H1 * psi_theta
print("Final State:")
print(psi_final)

## Plot Bloch Spheres

In [ ]:
State_List = [psi_initial, psi_x, psi_theta, psi_final]
for i, state in enumerate(State_List):
    # Bloch sphere
    #ax = fig.add_subplot(3,4,i+9)
    plt.figure()
    b = qutip.Bloch()
    b.font_size = 35
    b.vector_width = 8
    b.frame_width = 2
    b.size = [10,10]
    b.ylabel = [r'$y$','']
    b.zlabel = [r'$z = \left|0\right>$', r'$\left|1\right>$']
    b.add_states(state)
    b.zlpos = [1.25,-1.4]
    b.xlpos = [1.25,-1.25]
    b.ylpos = [1.2,-1.2]
    b.render()
    plt.savefig(f'OneQubit_Bloch_{i}.svg')


# Two Qubits (Un-entangled)

In [ ]:
from scipy.special.basic import h2vp
# intial state
psi_initial = ket("00")
print("Inital State:")
print(psi_initial)

# Rotate into x direction with Hadamard transformation
H2 = qutip.qip.operations.gates.hadamard_transform(2) #one qubit Hadamard
print("One qubit Hadamard:")
print(H2)

psi_x = H2 * psi_initial #Apply Hadamard by multiplication
print("\nResulting state:")
print(psi_x)

# Define Rz gate for theta = pi/3
phi = np.pi/3
Rz_phi = qutip.qip.operations.gates.rz(phi) #you could also just call rz(theta), but this way is more precise
Rz_phi = tensor(Rz_phi,Rz_phi)
print("Rz operator:")
print(Rz_phi)

# Apply Rz gate to inital state
psi_theta = Rz_phi * psi_x
print("\nRotated State:")
print(psi_theta)

# Action of final beamsplitter 
psi_final = H2 * psi_theta
print("Final State:")
print(psi_final)

In [ ]:
#define ranges for spherical plotting on the Bloch sphere
theta = np.linspace(0,np.pi,200)
phi = np.linspace(0,2*np.pi,200)

#Flip |0> and |1> for consistancy with definition in Wigner Function
XX_gate = (qutip.qip.operations.gates.x_gate(2,0)*qutip.qip.operations.gates.x_gate(2,1)).full

#Compute Wigner function and meshes for plotting later
W_initial, THETA, PHI = spin_wigner(symmetric_projection(Qobj(XX_gate()*psi_initial)), theta, phi)
W_psi_x, THETA, PHI = spin_wigner(symmetric_projection(Qobj(XX_gate()*psi_x)), theta, phi)
W_psi_theta, THETA, PHI = spin_wigner(symmetric_projection(Qobj(XX_gate()*psi_theta)), theta, phi)
W_final, THETA, PHI = spin_wigner(symmetric_projection(Qobj(XX_gate()*psi_final)), theta, phi)

Wigner_List = [W_initial, W_psi_x, W_psi_theta, W_final]

In [ ]:
# Plotting

# Cartisian Coordinates for sphere
x = np.cos(PHI-np.pi/2) * np.sin(THETA)
y = np.sin(PHI-np.pi/2) * np.sin(THETA) # cheating to make the axes match up...
z = np.cos(THETA)

# ring
t = linspace(0,2*np.pi,200)
x_ring = np.cos(t)
y_ring = np.sin(t)
z_ring = 0*t

def plot_grid(ax):
  ax.plot3D(x_ring,y_ring,z_ring,'black',zorder=20,linewidth=1,alpha=1)
  
  #Big lines
  ax.plot3D([0,1.3],[0,0],[0,0],'black',zorder=21,linewidth=2,alpha=1)
  ax.plot3D([0,0],[0,-1.6],[0,0],'black',zorder=22,linewidth=2,alpha=1)
  ax.plot3D([0,0],[0,0],[0,1.5],'black',zorder=23,linewidth=2,alpha=1)
  #small lines
  ax.plot3D([0,-1],[0,0],[0,0],'black',zorder=24,linewidth=1,alpha=1)
  ax.plot3D([0,0],[0,1],[0,0],'black',zorder=25,linewidth=1,alpha=1)
  ax.plot3D([0,0],[0,0],[0,-1],'black',zorder=26,linewidth=1,alpha=1)

# Set the aspect ratio to 1 so our sphere looks spherical
#fig = plt.figure(figsize=plt.figaspect(0.5))

for i, Wigner in enumerate(Wigner_List):
  fcolors = Wigner.real
  fmax, fmin = fcolors.max(), fcolors.min()
  #fcolors = (fcolors)/(2*np.max(fcolors))+0.5 
  fcolors = (fcolors)/(2*0.8)+0.5 

  fig = plt.figure(figsize=plt.figaspect(1))
  ax = fig.add_subplot(1,1,1, projection='3d')
  ax.plot_surface(x, y, z,  rstride=1, cstride=1, facecolors=cm.RdBu(fcolors))
  plot_grid(ax)
  ax.set_axis_off()
  fig.savefig(f'BlockSphere_2qubit_{i}.svg', transparent=True)



# Two Qubits (Entangled)

In [ ]:
from scipy.special.basic import h2vp
# intial state
psi_input = ket("00")
print("Input State:")
print(psi_input)

H1_X = tensor(qutip.qip.operations.gates.hadamard_transform(1),qutip.qip.operations.gates.x_gate(1))
CNOT = qutip.qip.operations.gates.cnot(2)
print("Hadamard x X:")
print(H1_X)

print("CNOT:")
print(CNOT)

psi_initial = CNOT * H1_X * psi_input
print("Inital State:")
print(psi_initial)


# Rotate into x direction with Hadamard transformation
H2 = qutip.qip.operations.gates.hadamard_transform(2) #one qubit Hadamard
print("One qubit Hadamard:")
print(H2)

psi_x = H2 * psi_initial #Apply Hadamard by multiplication
print("\nResulting state:")
print(psi_x)

# Define Rz gate for theta = pi/3
phi = np.pi/3
Rz_phi = qutip.qip.operations.gates.rz(phi) #you could also just call rz(theta), but this way is more precise
Rz_phi = tensor(Rz_phi,Rz_phi)
print("Rz operator:")
print(Rz_phi)

# Apply Rz gate to inital state
psi_theta = Rz_phi * psi_x
print("\nRotated State:")
print(psi_theta)

# Action of final beamsplitter 
psi_final = H2 * psi_theta
print("Final State:")
print(psi_final)

In [ ]:
#define ranges for spherical plotting on the Bloch sphere
theta = np.linspace(0,np.pi,200)
phi = np.linspace(0,2*np.pi,200)

#Flip |0> and |1> for consistancy with definition in Wigner Function
XX_gate = (qutip.qip.operations.gates.x_gate(2,0)*qutip.qip.operations.gates.x_gate(2,1)).full

#Compute Wigner function and meshes for plotting later
W_initial, THETA, PHI = spin_wigner(symmetric_projection(Qobj(XX_gate()*psi_initial)), theta, phi)
W_psi_x, THETA, PHI = spin_wigner(symmetric_projection(Qobj(XX_gate()*psi_x)), theta, phi)
W_psi_theta, THETA, PHI = spin_wigner(symmetric_projection(Qobj(XX_gate()*psi_theta)), theta, phi)
W_final, THETA, PHI = spin_wigner(symmetric_projection(Qobj(XX_gate()*psi_final)), theta, phi)

Wigner_List = [W_initial, W_psi_x, W_psi_theta, W_final]

In [ ]:
# Plotting

# Cartisian Coordinates for sphere
x = np.cos(PHI-np.pi/2) * np.sin(THETA)
y = np.sin(PHI-np.pi/2) * np.sin(THETA) # cheating to make the axes match up...
z = np.cos(THETA)

# ring
t = linspace(0,2*np.pi,200)
x_ring = np.cos(t)
y_ring = np.sin(t)
z_ring = 0*t

def plot_grid(ax):
  ax.plot3D(x_ring,y_ring,z_ring,'black',zorder=20,linewidth=1,alpha=1)
  
  #Big lines
  ax.plot3D([0,1.3],[0,0],[0,0],'black',zorder=21,linewidth=2,alpha=1)
  ax.plot3D([0,0],[0,-1.6],[0,0],'black',zorder=22,linewidth=2,alpha=1)
  ax.plot3D([0,0],[0,0],[0,1.5],'black',zorder=23,linewidth=2,alpha=1)
  #small lines
  ax.plot3D([0,-1],[0,0],[0,0],'black',zorder=24,linewidth=1,alpha=1)
  ax.plot3D([0,0],[0,1],[0,0],'black',zorder=25,linewidth=1,alpha=1)
  ax.plot3D([0,0],[0,0],[0,-1],'black',zorder=26,linewidth=1,alpha=1)

# Set the aspect ratio to 1 so our sphere looks spherical
#fig = plt.figure(figsize=plt.figaspect(0.5))

for i, Wigner in enumerate(Wigner_List):
  fcolors = Wigner.real
  fmax, fmin = fcolors.max(), fcolors.min()
  #fcolors = (fcolors)/(2*np.max(fcolors))+0.5 
  fcolors = (fcolors)/(2*0.8)+0.5 

  fig = plt.figure(figsize=plt.figaspect(1))
  ax = fig.add_subplot(1,1,1, projection='3d')
  ax.plot_surface(x, y, z,  rstride=1, cstride=1, facecolors=cm.RdBu(fcolors))
  plot_grid(ax)
  ax.set_axis_off()
  fig.savefig(f'BlockSphere_2qubit_Entangled_{i}.svg', transparent=True)